In [1]:
from solve_lindblad import *
import qutip as qu

from scipy.linalg import expm
import scipy

In [2]:
from derivatives import compute_probability

In [3]:
final_t = 1.2

omega = 10
J = 0.8
H0 = omega * qu.sigmax()
psi0 = qu.basis(2, 0)
rho0 = qu.ket2dm(psi0)

In [ ]:
%%timeit
evolve_dm(rho0, H0, [J * qu.sigmax()], final_t, 1)


In [4]:

def my_evolve_dm(rho0, H0, operators, t, no_qubits=1):
    liouvillian = qu.liouvillian(H0, c_ops=operators)
    rho_sup = qu.to_super(rho0)
    rho_evol = scipy.linalg.expm(liouvillian.full()*t)@rho_sup.full()
    dm_evol = rho_evol[:, 0].reshape(-1, 2**no_qubits)
    dm_evol = qu.Qobj(dm_evol)
    return dm_evol

In [ ]:
%%timeit
my_evolve_dm(rho0, H0, [J * qu.sigmax()], final_t, 1)


In [ ]:
a = qu.sigmax()

In [ ]:
%%timeit 
qu.sigmax()

In [ ]:
%%timeit 
qu.sigmax()

In [5]:
class a:
    def __init__(self):
        self.p = 2
    @property
    def piernas(self):
        return 2

In [6]:
b = a()

In [ ]:
%%timeit
b.piernas

In [ ]:
%%timeit
b.p

In [7]:
import custom_models


In [8]:
a = custom_models.SimplePrecessionWithNoise()

In [9]:
a.UnitH0

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 1.]
 [1. 0.]]

In [9]:
import numpy as np
from evolution import evolve_dm
import qutip as qu

In [10]:
rho = evolve_dm(a.SuperRho0, a.UnitH0, 0.3*qu.sigmax(), 0.3)

In [13]:
%%timeit
evolve_dm(a.SuperRho0, a.UnitH0, 0.3*qu.sigmax(), 0.3)

1.12 ms ± 50.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
for i in range(10):
    evolve_dm(a.SuperRho0, a.UnitH0, 0.3*qu.sigmax(), 0.3)

In [11]:
import cython
%load_ext Cython


In [ ]:
%%cython

from evolution import evolve_dm
def hola(int reps, *args):
    for i in range(reps):
        evolve_dm(*args)

In [ ]:
%%timeit
hola(10, a.SuperRho0, a.UnitH0, 0.3*qu.sigmax(), 0.3)

In [12]:

import derivatives
from evolution import Z_projs

In [13]:
import evolution

In [14]:
projs = [np.array([[1, 0], [0, 0]]), np.array([[0, 0], [0, 1]])]

In [ ]:
%%timeit
compute_probability(rho0.full(), projs)

In [ ]:
%%timeit
qu.measurement.measurement_statistics(rho0, qu.sigmaz())

In [15]:
p0 = compute_probability(rho0.full(), projs)

In [16]:
p0

array([1., 0.])

In [17]:
p0[0]

1.0

In [18]:
from qinfer.tests import test_model
from qinfer import UniformDistribution
from custom_models import SimplePrecessionWithNoise
import numpy as np


model = SimplePrecessionWithNoise()
prior = UniformDistribution([[0, 0.5], [0, 0.5]])
prior.sample()


modelparams = np.dstack(np.mgrid[0:1:10j, 0:1:10j]).reshape(-1, 2)


expparams = np.empty((9,), dtype=model.expparams_dtype)
expparams["t"] = np.dstack(np.mgrid[1:10] * np.pi / 2)


# test_model(model, prior, expparams)


In [19]:
import itertools
all_possible_exps = list(itertools.product(expparams["t"], modelparams))
# a.likelihood([0, 1], modelparams, expparams)

In [21]:
b = a.likelihood([0, 1], modelparams, expparams)

In [23]:
print(b)

[[[Delayed('single_qu_likelihood-295a2c55-a8a2-41e2-be52-f421101470dc')
   Delayed('single_qu_likelihood-3bc38c9a-beee-4ae1-b952-70431f5cd34b')
   Delayed('single_qu_likelihood-4c43aac5-6b55-4e49-8af5-0134c20a5dcc')
   ...
   Delayed('single_qu_likelihood-014fe69e-70a6-4ae2-882a-301ca4a0eb3d')
   Delayed('single_qu_likelihood-04728ca8-2b1b-41f0-90fe-c03aafc19818')
   Delayed('single_qu_likelihood-8f7416d3-7042-46fb-8688-4377dd780a92')]
  [Delayed('single_qu_likelihood-6718ace9-5b4f-4849-a209-12855b873062')
   Delayed('single_qu_likelihood-c6daf4e8-4b6f-4254-9fe5-6afa9ab715a4')
   Delayed('single_qu_likelihood-33816f2d-f412-4c4b-8d2f-340298bba46f')
   ...
   Delayed('single_qu_likelihood-d890811b-6320-44a4-80a5-d4a2cbf056bf')
   Delayed('single_qu_likelihood-45e92bd7-ef6c-4027-94d0-aabc4fafb814')
   Delayed('single_qu_likelihood-425dc50d-95f1-44d6-8e30-1b70f2db521b')]
  [Delayed('single_qu_likelihood-c12a76cf-3a37-46b6-80e8-931c5f14842f')
   Delayed('single_qu_likelihood-6c1dfbc6-0027-4

In [24]:
%%timeit
aux1 = qu.parallel_map(a.single_qu_likelihood, all_possible_exps)

429 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit
aux2 = qu.parfor(a.single_qu_likelihood, all_possible_exps)

318 ms ± 8.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
aux1 = qu.parallel_map(a.single_qu_likelihood, all_possible_exps)

In [27]:
aux2 = qu.parfor(a.single_qu_likelihood, all_possible_exps)

In [28]:
aux2 == aux1

True

In [ ]:
%%cython 

from cython.parallel import prange

def run(a, exps):
    re = []
    n = len(exps)

    for i in prange(n, nogil=True):
    # for i in exps:
        re.append(a.single_qu_likelihood(exps[i]))
    return re



# def func(double[:] x, double alpha):
#     cdef Py_ssize_t i

#     for i in prange(x.shape[0], nogil=True):
#         x[i] = alpha * x[i]    

In [ ]:
%%timeit
run(a, all_possible_exps)

In [29]:
all_possible_exps[0]

(1.5707963267948966, array([0., 0.]))

In [ ]:
from multiprocessing import Pool

In [41]:


%%timeit
re = []
with Pool() as pool:
    # call the same function with different data in parallel
    for result in pool.map(a.single_qu_likelihood, all_possible_exps):
        re.append(result)
        # report the value to show progress

323 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
re ==aux1

True

In [37]:
%%timeit
qu.parfor(a.single_qu_likelihood, all_possible_exps)

329 ms ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
from dask import delayed

import dask



In [31]:
%%timeit
re = []
for x in all_possible_exps:
    y = delayed(a.single_qu_likelihood)(x)
    re.append(y)

results = dask.compute(*re)


# with Pool() as pool:
#     # call the same function with different data in parallel
#     for result in pool.map(a.single_qu_likelihood, all_possible_exps):
#         re.append(result)
#         # report the value to show progress

1.25 s ± 48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
re = []
for x in all_possible_exps:
    y = delayed(a.single_qu_likelihood)(x)
    re.append(y)


In [30]:

dask.compute(*re)

(array(1.),
 array(0.98097871),
 array(0.92814748),
 array(0.85267334),
 array(0.76882123),
 array(0.68961216),
 array(0.62376006),
 array(0.5747492),
 array(0.54177767),
 array(0.52160696),
 array(0.96984631),
 array(0.95197214),
 array(0.90232703),
 array(0.83140454),
 array(0.75260933),
 array(0.67817715),
 array(0.61629642),
 array(0.57024127),
 array(0.53925817),
 array(0.5203039),
 array(0.88302222),
 array(0.86845107),
 array(0.82798),
 array(0.77016345),
 array(0.70592901),
 array(0.64525134),
 array(0.59480571),
 array(0.55726121),
 array(0.53200355),
 array(0.51655189),
 array(0.75),
 array(0.74048935),
 array(0.71407374),
 array(0.67633667),
 array(0.63441062),
 array(0.59480608),
 array(0.56188003),
 array(0.5373746),
 array(0.52088884),
 array(0.51080348),
 array(0.58682409),
 array(0.58352108),
 array(0.57434703),
 array(0.56124108),
 array(0.54668032),
 array(0.53292581),
 array(0.52149071),
 array(0.51298006),
 array(0.50725462),
 array(0.50375201),
 array(0.41317591),
